In [2]:
using LorentzianSimplexSolver;

In [ ]:
# ------------------------------------------------------------
# 1. Precision choice (user-controlled)
# ------------------------------------------------------------
const ScalarT = Float64
# const ScalarT = BigFloat

if ScalarT === BigFloat
    LorentzianSimplexSolver.PrecisionUtils.set_big_precision!(prec)
    LorentzianSimplexSolver.PrecisionUtils.set_tolerance!(sqrt(eps(BigFloat)))
else
    LorentzianSimplexSolver.PrecisionUtils.set_tolerance!(1e-10)
end

# ------------------------------------------------------------
# 2. Read simplices
# ------------------------------------------------------------
simplices = [[1,2,3,4,5], [1,2,4,5,6]]

ns = length(simplices)

all_vertices = unique(Iterators.flatten(simplices))
sort!(all_vertices)

Nverts = length(all_vertices)

# ------------------------------------------------------------
# 3. Read vertex coordinates
# ------------------------------------------------------------
vertex_coords = Dict{Int, Vector{ScalarT}}()    

coords_lines = [
    "0, 0, 0, 0",
    "0, 0, 0, 1",
    "0, 0, 1, 1",
    "0, 1, 1, 1",
    "1//2, 1, 1, 1",
    "0, 1, 0, 1",
]

for (v, line) in zip(all_vertices, coords_lines)
    vertex_coords[v] = LorentzianSimplexSolver.PrecisionUtils.parse_numeric_line(line, ScalarT)
end

In [ ]:
# ------------------------------------------------------------
# 4. Build geometry
# ------------------------------------------------------------
datasets = LorentzianSimplexSolver.GeometryTypes.GeometryDataset{ScalarT}[]

for (s, simplex) in enumerate(simplices)
    println("\n--- Processing simplex $s with vertices $simplex ---")
    bdypoints = [vertex_coords[v] for v in simplex]
    ds = LorentzianSimplexSolver.GeometryPipeline.run_geometry_pipeline(bdypoints)
    push!(datasets, ds)
end

geom = LorentzianSimplexSolver.GeometryTypes.GeometryCollection(datasets);


--- Processing simplex 1 with vertices [1, 2, 3, 4, 5] ---

--- Processing simplex 2 with vertices [1, 2, 4, 5, 6] ---


In [9]:
# ------------------------------------------------------------
# 5. Consistency checks for each simplex
# ------------------------------------------------------------
for (idx, simplex) in enumerate(geom.simplex)
    LorentzianSimplexSolver.GeometryConsistency.check_sl2c_parallel_transport(simplex.solgsl2c, simplex.bdybivec55)
    LorentzianSimplexSolver.GeometryConsistency.check_so13_parallel_transport(simplex.solgso13, simplex.bdybivec4d55)
    LorentzianSimplexSolver.GeometryConsistency.check_closure_bivectors(simplex.kappa, simplex.areas, simplex.bdybivec55)
end

✓ SL(2,C) parallel transport satisfied (max residual = 2.7755575615628914e-16).
✓ SO(1,3) parallel transport satisfied (max residual = 5.266250202865052e-16)
✓ Closure condition satisfied for bivectors (max residual = 8.777083671441753e-17)
✓ SL(2,C) parallel transport satisfied (max residual = 2.7755575615628914e-16).
✓ SO(1,3) parallel transport satisfied (max residual = 5.266250202865052e-16)
✓ Closure condition satisfied for bivectors (max residual = 8.777083671441753e-17)


In [ ]:
# ------------------------------------------------------------
# 6. Connect simplices + face matching + gauge fixing
# ------------------------------------------------------------
if ns > 1
    println("\nFixing global κ-sign orientation ...")
    LorentzianSimplexSolver.KappaOrientation.fix_kappa_signs!(simplices, geom)

    println("\nBuilding global connectivity ...")
    conn = LorentzianSimplexSolver.FourSimplexConnectivity.build_global_connectivity(simplices, geom)
    push!(geom.connectivity, conn)

    LorentzianSimplexSolver.FaceXiMatching.run_face_xi_matching(geom)
    println("Global connectivity constructed.")

    # println("\nRunning final face-matching checks ...")
    LorentzianSimplexSolver.FaceMatchingChecks.check_all(geom)

    println("\nPerform SU(2) and SU(1,1) gauge fixing ...")
    LorentzianSimplexSolver.GaugeFixingSU.run_su2_su11_gauge_fix(geom)
    println("\nGauge fixing finished.")
end



Fixing global κ-sign orientation ...
[1, 2]

Building global connectivity ...
Global connectivity constructed.

Running face-ξ matching ...
  SL(2,C) matrices updated:      ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓


Running final face-matching checks ...

   FACE–XI GEOMETRY CHECKS

✓ Face matching satisfied (max residual = 8.144890172189112e-16).
✓ SL(2,C) parallel transport satisfied (max residual = 1.3427790637090416e-15).
✓ Closure condition satisfied (max residual = 1.1775693440128312e-16).

------------------------------
✓ All geometric consistency checks PASSED.
------------------------------


Perform SU(2) and SU(1,1) gauge fixing ...

Gauge fixing finished.


In [ ]:
# ------------------------------------------------------------
# 7. Symbols, action
# ------------------------------------------------------------
LorentzianSimplexSolver.DefineSymbols.run_define_variables(geom)

sd, _ = LorentzianSimplexSolver.SolveVars.run_solver(geom)

S = LorentzianSimplexSolver.DefineAction.compute_action(geom)

S_fn, labels = LorentzianSimplexSolver.SymbolicToJulia.build_action_function(S, sd)

@variables γ
args = LorentzianSimplexSolver.SymbolicToJulia.build_argument_vector(sd, labels, γ)
S_sym = simplify(S_fn(args...)); 

In [ ]:
# ------------------------------------------------------------
# 8. EOMs
# ------------------------------------------------------------
dS = LorentzianSimplexSolver.EOMsHessian.compute_EOMs(S, sd)

grad_fns = LorentzianSimplexSolver.SymbolicToJulia.build_gradient_functions(dS, sd)

LorentzianSimplexSolver.EOMsHessian.check_EOMs(grad_fns, sd; γ = one(ScalarT));

In [ ]:
# ------------------------------------------------------------
# 9. Hessian computations
# ------------------------------------------------------------
H = LorentzianSimplexSolver.EOMsHessian.compute_Hessian(S, sd)

hess_fns = LorentzianSimplexSolver.SymbolicToJulia.build_hessian_functions(H, sd)
H_eval, _ = LorentzianSimplexSolver.EOMsHessian.evaluate_hessian(hess_fns, sd; γ = one(ScalarT))

H_det = det(H_eval)